<a href="https://colab.research.google.com/github/RedsquareM/Project1/blob/main/RockPaperAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# Устанавливаем kaggle environment чтобы оценить эпизоды
!pip install -q -U kaggle_environments

In [46]:
# Импортируем библиотеки
# numpy - библиотека для arrays, быстрота операций 
# pandas - библиотека для дата анализа 
# matplot lib - анимация и визуаизация боев 
# seaborn - анимация, статистика 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [47]:
# создаем площaдку для созтязания агентов 
env = make("rps",
      configuration = {
          "episodeSteps":100
      }
)

Условия игры каждый агент будет руководится следующими правилами:
Если "0" то камень (rock) 
Если "1" то бумага (paper)
Если "2" то ножницы (scissors)

Правила: 
1. Камень побеждает ножницы
2. Ножницы побеждают бумагу
3. Бумага побеждает камень 

Ничья при одинаковых выбросов

In [48]:
%%writefile only_rock.py
# Первый агент который только использует стратегию выбрасывания камня 

def only_rock(observation, configuration):
    return 0

Overwriting only_rock.py


In [112]:
%%writefile only_scissors.py
# Второй агент который выбрасивает только ножницы 
def only_scissors(observation, configuration):
  return 2

Overwriting only_scissors.py


In [50]:
%%writefile only_paper.py
# Tретий агент который выбрасивает только бумагу
def only_paper(observation, configuration):
  return 1

Overwriting only_paper.py


In [51]:
%%writefile random.py
# Fourth agent that throws random choices of rock, paper, scissors
# Proability of each one of the choices is 1/3 

import random


def equilibrium_agent(observation, configuration):
    return random.randint(0, 2)

Overwriting random.py


In [52]:
%%writefile paper_rock.py
# Fifth agent that throws that chooses either rock or paper 
# The choice is either "0" for rock or "1" for paper 

import random

def paper_rock_agent(observation, configuration):
    return random.randint(0, 1)

Overwriting paper_rock.py


In [53]:
%%writefile paper_scissors.py
# Sixth agent that throws that chooses either rock or paper 
# The choice is either "1" paper or "2" scissors 
import random

def paper_scissors_agent(observation, configuration):
    return random.randint(1, 2)



Overwriting paper_scissors.py


In [54]:
%%writefile rock_scissors.py
# Seventh agent that throws either rock or scissors
# The choice is either "0" rock or "2" scissors 


import random

def main():
# Defined two possible outcomes 
  def only_rock():
    return 0
  def only_scissors():
    return 2
#use the list and random choice between two possibilities
  mylist = [only_rock, only_scissors]
  return random.choice(mylist)()


Writing rock_scissors.py


In [55]:
%%writefile copy.py
# Agent # 8 will copy the opponent choice 

import numpy as np
import random

# Checks what was opponent throw and returns the same value 
def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return 0


Overwriting copy.py


In [56]:
%%writefile copy_with_step.py
# Agent # 9 will copy the opponent choice and add +1 to it 
# If opponent throws a rock (0) then our agent will throw a paper(1)

import numpy as np
import random

def copy_with_step(observation, configuration):
    if observation.step > 0:
        opp_hand = observation.lastOpponentAction
        return (opp_hand + 1) % 3 
    else:
        return 1

  




Overwriting copy_with_step.py


In [57]:
%%writefile mixagent10.py
#Agent #10 will throw in first 9 rounds a rock
#From rounds 10 to 19 will throw only paper
# and the rest of the rounds will throw scissors

import numpy as np
import random
import pandas as pd 

# Checks what was opponent throw and returns the same value 
def copy_opponent_agent(observation, configuration):
    if 0 < observation.step < 10:
        #opp_hand = observation.lastOpponentAction
        return 0
    elif 10 < observation.step < 20:
        #opp_hand = observation.lastOpponentAction
        return 1
    elif observation.step > 20:
        #opp_hand = observation.lastOpponentAction
        return 2 

Overwriting mixagent10.py


In [58]:
%%writefile mixagent11.py
#Agent #11 will throw different outcomes as the rounds progress 
#Modification from other not just throwing a copy of the opponent 
#but adding a value to know opponent value 
#the variety of observation step can be manipulated to create more agents 

import numpy as np
import random
import pandas as pd 

# Checks what was opponent throw and returns the same value 
def copy_opponent_agent(observation, configuration):
    if 0 < observation.step < 10:
        opp_hand = observation.lastOpponentAction
        return (opp_hand + 2) % 3 
    elif 10 < observation.step < 50:
        opp_hand = observation.lastOpponentAction
        return (opp_hand) % 3
    elif observation.step > 50:
        opp_hand = observation.lastOpponentAction
        return (opp_hand -1) % 3 

Overwriting mixagent11.py


In [145]:
%%writefile my_options.py
# agent #12 will throw random choices based on predetermined options
# the number of choices varies between each choice but it is predetermined 

import random


def options(observation, configuration):
  my_options = [0]*40 + [1]*20 + [2]*30
  option_result = int(random.choice(my_options))
  return option_result 


    

Writing my_options.py


In [192]:
%%writefile myoptions_13.py
#Agent 13 uses different mix of strategies during the different rounds 
# in the first half of the rounds using copy with the step strategy
# in the middle rounds using random strategy
# and final rounds just copies last opponent action 

import random
import numpy as np

my_action = 0
my_options =(0,1,2)

def options13(observation, configuration):
    global my_action
    if 0< observation.step < 15:
      my_action = (my_action + 1) % 3
    elif 15 < observation.step < 60: 
      my_action = int(random.choice(my_options))
    elif observation.step > 60:
      my_action = observation.lastOpponentAction
    return my_action

    
 




Overwriting myoptions_13.py


In [184]:
%%writefile reactionary.py
# Agent 14 throws last reaction of the opponent
# If loses than adds + 1, import get_score helps in this case 

import random
from kaggle_environments.envs.rps.utils import get_score

last_react_action = None


def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Overwriting reactionary.py


In [60]:
%%writefile decision_tree.py
#this Agent #15 is complex algorithm that was taken from Kaggle 
#for information purpose to see how it plays agains my own agents 

import numpy as np
import collections
from sklearn.tree import DecisionTreeClassifier

def construct_local_features(rollouts):
    step_mode_features = np.array([[step % k for step in rollouts['steps']] for k in (2, 3, 5)])
    step_div_features = np.array([[step // k for step in rollouts['steps']] for k in (100, 150, 250)])
    features = np.concatenate([step_mode_features, step_div_features])
    features = np.append(features, rollouts['actions'])
    features = np.append(features, rollouts['opp-actions'])
    return features

def construct_global_features(rollouts):
    features = []
    for key in ['actions', 'opp-actions']:
        for i in range(3):
            actions_count = np.mean([r == i for r in rollouts[key]])
            features.append(actions_count)
    return np.array(features)

def construct_features(short_stat_rollouts, long_stat_rollouts):
    lf = construct_local_features(short_stat_rollouts)
    gf = construct_global_features(long_stat_rollouts)
    features = np.concatenate([lf, gf])
    return features

def predict_opponent_move(train_data, test_sample):
    classifier = DecisionTreeClassifier(random_state=42)
    classifier.fit(train_data['x'], train_data['y'])
    return classifier.predict(test_sample)

def update_rollouts_hist(rollouts_hist, last_move, opp_last_action):
    rollouts_hist['steps'].append(last_move['step'])
    rollouts_hist['actions'].append(last_move['action'])
    rollouts_hist['opp-actions'].append(opp_last_action)
    return rollouts_hist

def warmup_strategy(observation, configuration):
    global rollouts_hist, last_move
    action = int(np.random.randint(3))
    if observation.step == 0:
        last_move = {'step': 0, 'action': action}
        rollouts_hist = {'steps': [], 'actions': [], 'opp-actions': []}
    else:
        rollouts_hist = update_rollouts_hist(rollouts_hist, last_move, observation.lastOpponentAction)
        last_move = {'step': observation.step, 'action': action}
    return int(action)

def init_training_data(rollouts_hist, k):
    for i in range(len(rollouts_hist['steps']) - k + 1):
        short_stat_rollouts = {key: rollouts_hist[key][i:i+k] for key in rollouts_hist}
        long_stat_rollouts = {key: rollouts_hist[key][:i+k] for key in rollouts_hist}
        features = construct_features(short_stat_rollouts, long_stat_rollouts)        
        data['x'].append(features)
    test_sample = data['x'][-1].reshape(1, -1)
    data['x'] = data['x'][:-1]
    data['y'] = rollouts_hist['opp-actions'][k:]
    return data, test_sample

def agent(observation, configuration):
    # hyperparameters
    k = 5
    min_samples = 25
    global rollouts_hist, last_move, data, test_sample
    if observation.step == 0:
        data = {'x': [], 'y': []}
    # if not enough data -> randomize
    if observation.step <= min_samples + k:
        return warmup_strategy(observation, configuration)
    # update statistics
    rollouts_hist = update_rollouts_hist(rollouts_hist, last_move, observation.lastOpponentAction)
    # update training data
    if len(data['x']) == 0:
        data, test_sample = init_training_data(rollouts_hist, k)
    else:        
        short_stat_rollouts = {key: rollouts_hist[key][-k:] for key in rollouts_hist}
        features = construct_features(short_stat_rollouts, rollouts_hist)
        data['x'].append(test_sample[0])
        data['y'] = rollouts_hist['opp-actions'][k:]
        test_sample = features.reshape(1, -1)
        
    # predict opponents move and choose an action
    next_opp_action_pred = predict_opponent_move(data, test_sample)
    action = int((next_opp_action_pred + 1) % 3)
    last_move = {'step': observation.step, 'action': action}
    return action




Writing decision_tree.py


In [61]:
%%writefile markov.py
#Agent #16 another agent that was taken from Kaggle 
# This is very well know algorithm of Markov chains 
# Compare with other bots 

import numpy as np
import collections
import pandas as pd
from kaggle_environments import make, evaluate


def markov_agent(observation, configuration):
    k = 4
    global table, action_seq
    if observation.step % 250 == 0: # refresh table every 250 steps
        action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])    
    if len(action_seq) <= 2 * k + 1:
        action = int(np.random.randint(3))
        if observation.step > 0:
            action_seq.extend([observation.lastOpponentAction, action])
        else:
            action_seq.append(action)
        return action
    # update table
    key = ''.join([str(a) for a in action_seq[:-1]])
    table[key][observation.lastOpponentAction] += 1
    # update action seq
    action_seq[:-2] = action_seq[2:]
    action_seq[-2] = observation.lastOpponentAction
    # predict opponent next move
    key = ''.join([str(a) for a in action_seq[:-1]])
    if observation.step < 500:
        next_opponent_action_pred = np.argmax(table[key])
    else:
        scores = np.array(table[key])
        next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum()) # add stochasticity for second part of the game
    # make an action
    action = (next_opponent_action_pred + 1) % 3
    # if high probability to lose -> let's surprise our opponent with sudden change of our strategy
    if observation.step > 900:
        action = next_opponent_action_pred
    action_seq[-1] = action
    return int(action)

Writing markov.py


In [191]:
# This is the battle ground between two bots 
env.reset()
env.run(['only_rock.py', 'myoptions_13.py'])
env.render(mode="ipython", width=500, height=450)

In [ ]:
# Showing the results of the battle 

evaluate(
    "rps", #environment to use - no need to change
    ['markov.py', 'mixagent3.py'], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)



[[0, 0]]

In [204]:

# This is the tournament of my favorite agent that will play against all other agents 
# that were created in this task
# results are displayed 
# После многих соревнований разные стратегии хорошо работают против одних, но очень
# плохо против других, randomagent показал более или менее среднии результаты


from kaggle_environments import make
import random

def randomAgent(obs, config):
    return random.randint(0, 2)

def only_rock(obs,config):
    return 0

def only_paper(obs, config):
    return 1 

def only_scissors(obs, config):
    return 2

env = make('rps', debug = True)

# Агент которого используем против других 
def evaluate(copy_with_step):
    
    pool = [
        only_rock,
        only_paper,
        only_scissors,
        randomAgent,
        
    ]


    print()
    overall = 0

    for opponent in pool:

        env.reset()

        env.run([copy_with_step, opponent])
        json = env.toJSON()
        rewards = json['rewards']

        opponentName = str(opponent).split()[1].capitalize()
        padding = ''; length = len(str(rewards))
        if length < 15: padding = ' ' * (15 - length)

        print(f'Rewards: {rewards}{padding} copy_with_step vs {opponentName}')
        overall += rewards[0]
    
    overall /= len(pool)
    print(f'Overall Score: {overall}')
    
    return overall

In [205]:
score = evaluate('copy_with_step.py')
if score <= 0:
    print('\nUРезультат: Так себе стратегия')
else:
    print('\nРезультат: Больше побед чем поражений')


Rewards: [999.0, -999.0] copy_with_step vs Only_rock
Rewards: [998.0, -998.0] copy_with_step vs Only_paper
Rewards: [997.0, -997.0] copy_with_step vs Only_scissors
Rewards: [-23.0, 23.0]   copy_with_step vs Randomagent
Overall Score: 742.75

Результат: Больше побед чем поражений
